In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


# Charger votre jeu de données
data = pd.read_csv('data1.csv', delimiter=';', decimal='.')
data_pop = pd.read_csv('Communes.csv', delimiter=';', decimal='.')

#recuperation des colonnes interessantes
data = data[["Synthese_eval_sanit", "APP_Libelle_activite_etablissement", "Code_postal", "Libelle_commune"]]
data_pop = data_pop.drop(['PMUN','PCAP','DEPCOM'],axis=1)

#renommer la colonne code postal
data_pop.rename(columns={'COM': 'Libelle_commune'}, inplace=True)

#Fusion des df data et data_pop
df_fusionne = pd.merge(data,data_pop, on=['Libelle_commune'])

# Initialisez l'encodeur LabelEncoder
label_encoder = LabelEncoder()

# Transformez les catégories en valeurs numériques
df_fusionne["APP_Libelle_activite_etablissement"] = label_encoder.fit_transform(df_fusionne['APP_Libelle_activite_etablissement'])
df_fusionne["Libelle_commune"] = label_encoder.fit_transform(df_fusionne['Libelle_commune'])

# Conversion des évaluations en valeurs numériques ordinales
df_fusionne['Synthese_eval_sanit'] = df_fusionne['Synthese_eval_sanit'].map({'Très satisfaisant': 1, 'Satisfaisant': 2, 'A améliorer': 3,'A corriger de manière urgente':4})
#data['Date_inspection'] = pd.to_numeric(data['Date_inspection'], errors='coerce')


X = df_fusionne[["APP_Libelle_activite_etablissement", "Libelle_commune","PTOT"]]
y = df_fusionne['Synthese_eval_sanit']

# Division des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardisation des caractéristiques (peut ne pas être nécessaire selon les prédicteurs)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Création du modèle de régression logistique ordinale
model = LogisticRegression(multi_class='ovr', solver='liblinear')  # 'ovr' pour la régression ordonnée

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.547836617857536
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00      3438
           2       0.55      1.00      0.71      4976
           3       0.00      0.00      0.00       635
           4       0.00      0.00      0.00        34

    accuracy                           0.55      9083
   macro avg       0.14      0.25      0.18      9083
weighted avg       0.30      0.55      0.39      9083



/home/heartless/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heartless/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heartless/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
# test du modele Random Forest Classifier
clf = RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

#Score random forest classifier
acc = accuracy_score(y_test,y_pred)
acc

0.5946273257734229

In [39]:
#enregistrement des resultats dans un fichier csv
df_fusionne.to_csv('Data_finale1.csv', index=False,sep=';')